In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np
import uuid        # for id generation

import chess.pgn
from stockfish import Stockfish

In [3]:
# stockfish = Stockfish(
#     'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#     parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
# )
# stockfish.set_elo_rating(2600)
# stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'White': [],  # Dummy ID
    'Black': [],  # Dummy ID
    'ECO': [],
    'FEN_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'Result': [],
}

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # Moves
        fen_pos = []
        acpl = []
        
        moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])  # dummy ID for this dataset only breaks
        moves_log_dict['White'].append(game.headers['White'])  # dummy ID
        moves_log_dict['Black'].append(game.headers['Black'])  # dummy ID
        moves_log_dict['ECO'].append(game.headers['ECO'])
        moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        moves_log_dict['Result'].append(game.headers['Result'])

        # MOVE CYCLE
        for move in moves:
            board.push(move)
            fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
#             cpl = stockfish.get_evaluation()['value']/100
#             acpl.append(cpl)
        
        moves_log_dict['FEN_moves'].append(fen_pos)
        #moves_log_dict['cpl'].append(acpl)
        
        game_counter += 1
        if game_counter == 55:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

55 games read.
CPU times: user 291 ms, sys: 0 ns, total: 291 ms
Wall time: 291 ms


In [5]:
#generates unique IDs
def id_generator(id):
    return uuid.uuid4().int

## df_players

In [6]:
df_players = pd.DataFrame(players)
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [7]:
#pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8') 

In [8]:
#game = chess.pgn.read_game(pgn)

In [9]:
# def players_id_list(input_df, c_black, c_white): # c_white, c_black - columns of respective df, must be a string
#     black = input_df[c_black]
#     white = input_df[c_white]
#     players_names = list(set(list(black) + list(white)))
#     df_pl = {'Players': players_names}
#     players_id = pd.DataFrame(df_pl)
#     players_id['Player_ID'] = players_id['Players'].apply(id_generator)
#     return players_id

#download list? input_df.to_csv("/notebooks/data/players_id")

In [10]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids 
    return players_id

In [11]:
players_id = players_id_list(df_players, players_id)

In [12]:
players_id

,Player_ID,Players
0,235027232128446844895773802489407883991,iaquinuchess
1,76888196776999652768598745028210225020,JMM
2,252017249120170133210252650222868783350,SpeckEngine
3,3641273092479788963048870545248261231,forlat
4,216239674017034059556868881884327100084,chessloon
5,294575331559394627767875470153671346802,chenqisheng
6,184411655916863417189948281767136661682,sparse
7,223328724874470753875670859213651463632,exeComp
8,54201587794443287544394463773094020721,RamMiguel
9,143259623424430264343080445357975154516,Qiyas


In [13]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [14]:
df_dummy

,White,Black
0,12345,DummyName
1,DummyName,12345
2,1234,Dummy
3,forlat,Geforce
4,Geforce,Bambi


In [15]:
players_id  = players_id_list(df_dummy, players_id)

In [16]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players'])
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
    df_players = m_bw
    return df_players

In [17]:
assign_player_id(df_dummy)

,White,Black,White_ID,Black_ID
0,12345,DummyName,197888331855699428866524586342289299634,112370278862272705457153558247582697033
1,DummyName,12345,112370278862272705457153558247582697033,197888331855699428866524586342289299634
2,1234,Dummy,57107001729067977884612938887002162221,60819134570165239280037100779163346082
3,forlat,Geforce,3641273092479788963048870545248261231,252523479098315469581487124264000465768
4,Geforce,Bambi,252523479098315469581487124264000465768,58708882015197233749114640080441047813


In [18]:
# Fix it:
def update_and_assign_player_IDs(input_df, players_id):
    df_players = assign_player_id(input_df)
    return df_players

In [19]:
update_and_assign_player_IDs(df_dummy, players_id)

,White,Black,White_ID,Black_ID
0,12345,DummyName,197888331855699428866524586342289299634,112370278862272705457153558247582697033
1,DummyName,12345,112370278862272705457153558247582697033,197888331855699428866524586342289299634
2,1234,Dummy,57107001729067977884612938887002162221,60819134570165239280037100779163346082
3,forlat,Geforce,3641273092479788963048870545248261231,252523479098315469581487124264000465768
4,Geforce,Bambi,252523479098315469581487124264000465768,58708882015197233749114640080441047813


## df_games

In [20]:
df_games = pd.DataFrame(games)
df_games['Game_ID'] = df_games['Game_ID'].apply(id_generator)
##TODO date to datetime
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,91490709111883032979551203535023475339,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,118665393672501900023036510814820180788,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,103241585538818007258028342507097804922,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,85039859303235101259769755722991249201,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,251146080301942040863698818989306967268,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,279085020194123616791011415719792248149,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,282614773284892355342702573630838866209,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,301419612530200253199865255890561410499,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,48214567498279667263052271528253016993,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,300949113041299219439209051544271103682,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [21]:
timestamp = game.headers["Time"]

In [22]:
def game_id(df_games):
    df_games['Game_ID'] = df_games['Game_ID'].apply(id_generator)
    return df_games

In [23]:
game_id(df_games)

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,140980544747546788134209208999098552832,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,217174180498958476673080934492014949205,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,323123500757650799393808170528745316008,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,148428796080318513863853830864730953372,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,114936877647126254794397911897047621751,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,205480408292270186751180691334158230222,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,307896390889351547085483888271197164902,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,13238412079570168421506835805020403938,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,187960054030726211035583047453324962803,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,186359331497893488678152213623702792118,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [24]:
timestamp[:]

'13:55:00'

## df_moves

In [25]:
df_moves = pd.DataFrame(moves_log_dict)
#TODO moves and cpl as features when loading to DB
df_moves['Game_ID'] = df_games['Game_ID']
df_moves.head(20)

,Game_ID,White,Black,ECO,FEN_moves,WhiteIsComp,Result
0,140980544747546788134209208999098552832,forlat,Geforce,A01,[rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR...,Yes,0-1
1,217174180498958476673080934492014949205,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
2,323123500757650799393808170528745316008,forlat,Geforce,A20,[rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR...,Yes,0-1
3,148428796080318513863853830864730953372,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
4,114936877647126254794397911897047621751,forlat,Geforce,D30,[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...,Yes,1/2-1/2
5,205480408292270186751180691334158230222,exeComp,Ruvarashe,B92,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,1-0
6,307896390889351547085483888271197164902,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
7,13238412079570168421506835805020403938,forlat,Geforce,C28,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,0-1
8,187960054030726211035583047453324962803,Ruvarashe,exeComp,C11,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
9,186359331497893488678152213623702792118,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0


# search

In [26]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')